In [3]:
import pandas as pd

In [4]:
## Using google colab to run this code
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## GPU information
!nvidia-smi

Fri Oct 22 03:05:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# New Section

In [6]:
## huggingface.co: How to build bert model
!pip install transformers
## transformers
## (formerly called PyTorch - Transformers and PyTorch - Pretrained - Bert)
## General BERT family constructs for natural language understanding (NLU) 
## and natural language generation (NLG) (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet, etc.)
## contains more than 32 pre-training models covering more than 100 languages.

     |████████████████████████████████| 2.9 MB 2.7 MB/s 
     |████████████████████████████████| 56 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import numpy as np
import torch 
import torch.nn as nn
import sys
import shutil

In [8]:
## load the data
df = pd.read_csv("drive/MyDrive/train_data_labels.csv")
df

,train_id,abstract,label
0,1,As a new kind of two-dimensional carbon allotr...,Composites
1,2,"A.S.T.M. Standards, 1949. 6 volumes. Over 8,00...",Vinyl
2,3,Single-photon photoionization mass spectrometr...,Particulate matter
3,4,The molecular and crystal structure of the hyd...,Crystal structure
4,5,"The first [2]silacobaltocenophane, [(MeSi)(η-C...",Crystal structure
...,...,...,...
89995,89996,We study the scattering properties of coated n...,Insulators
89996,89997,We describe the characterization of the hydrod...,Particulate matter
89997,89998,The P2-type layered oxide Na[NiMn]O cathode ma...,Coating materials
89998,89999,We proposed the concept of atomic layer modula...,Thin films


In [9]:
## one-hot encoding
y = pd.get_dummies(df['label'])
 ## Horizontal table splicing (row alignment)
df = pd.concat([df,y],axis=1)
## remove id and label
df = df.drop(['train_id','label'],axis=1)
df

,abstract,Alloys,Anode materials,Coating materials,Composites,Crystal structure,Crystals,Dyes and pigments,Electrical conductivity,Fibers,Foams,Granular materials,Insulators,Layered materials,Particulate matter,Plastics,Porous materials,Solvents,Thin films,Vinyl
0,As a new kind of two-dimensional carbon allotr...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"A.S.T.M. Standards, 1949. 6 volumes. Over 8,00...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Single-photon photoionization mass spectrometr...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,The molecular and crystal structure of the hyd...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The first [2]silacobaltocenophane, [(MeSi)(η-C...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,We study the scattering properties of coated n...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
89996,We describe the characterization of the hydrod...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
89997,The P2-type layered oxide Na[NiMn]O cathode ma...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
89998,We proposed the concept of atomic layer modula...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [10]:
## labels
y_list = y.columns
y_list

Index(['Alloys', 'Anode materials', 'Coating materials', 'Composites',
       'Crystal structure', 'Crystals', 'Dyes and pigments',
       'Electrical conductivity', 'Fibers', 'Foams', 'Granular materials',
       'Insulators', 'Layered materials', 'Particulate matter', 'Plastics',
       'Porous materials', 'Solvents', 'Thin films', 'Vinyl'],
      dtype='object')

In [11]:
# hyper-parameter
max_len = 256 # Maximum sentence length: How many dimensions
train_batch_size = 5 # Parameters are updated every 4 samples
valid_batch_size = 5
epochs = 3 ## training times
learning_rate = 0.00001 

In [12]:
from transformers import BertTokenizer, BertModel

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [14]:
class PreProcessingDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['abstract']
        self.targets = self.df[y_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        ## feature extraction
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True, ## speical_tokens
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt' ## The type of data returned, tf ', 'pt' or 'NP', tF.constant, Torch.Tensor NP. Ndarray
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [15]:
# df = df[:300]
## split the data into train dataset and validation dataset
train_size_split = 0.8
train_df = df.sample(frac=train_size_split, random_state=200) # random_state: random seed
val_df = df.drop(train_df.index)
train_df=train_df.reset_index(drop=True)# re-shuffle the data
val_df=val_df.reset_index(drop=True)

In [16]:
## pre-processing
train_dataset = PreProcessingDataset(train_df, tokenizer, max_len)
valid_dataset = PreProcessingDataset(val_df, tokenizer, max_len)

In [17]:
## Batch size samples are extracted from the database each time
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size = train_batch_size,
    shuffle=True # set to ``True`` to have the data reshuffled at every epoch (default: False).
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=valid_batch_size,
    shuffle=False
)

In [19]:
## Using GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [20]:
#for saving and loading models
def load_ckp(checkpoint_fpath, model, optimizer):
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [21]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        ## bert-base-uncased: 12-layer, 768-hidden, 12-heads, 110M parameters
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.5) ## prevent overfitting (default 0.5)
        self.linear = torch.nn.Linear(768, 19)### 19 output class numbers
    
    ## forward caculation
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)
# model.train()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [22]:
## loss function
def loss_function(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=learning_rate)

In [24]:
val_targets=[]
val_outputs=[]

In [25]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train() # prep model for training

    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        # print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids) ## forward pass: compute predicted outputs by passing inputs to the model
        optimizer.zero_grad() ## The gradient is zero
        loss = loss_function(outputs, targets) ## caculate the loss

        ## print the loss
        if batch_idx%50==0:
           print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad() ## The gradient is zero
        loss.backward() ## Back propagation is calculated to obtain the gradient value of each parameter
        optimizer.step() ## Gradient descent performs a parameter update

        # print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        # print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))


    ######################    
    # validate the model #
    ######################
 

    model.eval() # prep model for evaluation
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_function(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/(len(training_loader)+1)
      valid_loss = valid_loss/(len(validation_loader)+1)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [26]:
ckpt_path = "/content/drive/MyDrive/test_final2"
best_model_path = "/content/drive/MyDrive/best_final_model2.pt"

In [27]:
trained_model = train_model(epochs, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)


############# Epoch 1: Training Start   #############
Epoch: 1, Training Loss:  0.6950708031654358
Epoch: 1, Training Loss:  0.430818647146225
Epoch: 1, Training Loss:  0.31123456358909607
Epoch: 1, Training Loss:  0.26710620522499084
Epoch: 1, Training Loss:  0.19948674738407135
Epoch: 1, Training Loss:  0.24066339433193207
Epoch: 1, Training Loss:  0.23171861469745636
Epoch: 1, Training Loss:  0.22463078796863556
Epoch: 1, Training Loss:  0.19222772121429443
Epoch: 1, Training Loss:  0.23003199696540833
Epoch: 1, Training Loss:  0.20401282608509064
Epoch: 1, Training Loss:  0.1584549844264984
Epoch: 1, Training Loss:  0.18306678533554077
Epoch: 1, Training Loss:  0.17559367418289185
Epoch: 1, Training Loss:  0.2016841322183609
Epoch: 1, Training Loss:  0.15260499715805054
Epoch: 1, Training Loss:  0.20173613727092743
Epoch: 1, Training Loss:  0.20093239843845367
Epoch: 1, Training Loss:  0.19408293068408966
Epoch: 1, Training Loss:  0.19343315064907074
Epoch: 1, Training Loss:  0.189

In [28]:
trained_model

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [29]:
trained_model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [30]:
val_df

,abstract,Alloys,Anode materials,Coating materials,Composites,Crystal structure,Crystals,Dyes and pigments,Electrical conductivity,Fibers,Foams,Granular materials,Insulators,Layered materials,Particulate matter,Plastics,Porous materials,Solvents,Thin films,Vinyl
0,The molecular and crystal structure of the hyd...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The first [2]silacobaltocenophane, [(MeSi)(η-C...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rational drug design depends on the knowledge ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Electrophoretically induced aggregation of Au ...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,"Dielectric relaxation of the ionic liquid, 1-e...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,The rehydration of thermoresponsive poly(monom...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
17996,The experimental solubility data of fenofibrat...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
17997,"Niobium pentoxide, NbO, exists in many polymor...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
17998,Pure and -nitroaniline (O-NA) doped benzil sin...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
final_output[0].index(max(final_output[0]))

4

In [33]:
inx_class_dict=dict(zip(range(0,len(y_list)),list(y_list)))

In [34]:
inx_class_dict[final_output[0].index(max(final_output[0]))]

'Crystal structure'

In [35]:
def predict(model,input):
  encodings = tokenizer.encode_plus(
    input,
    None,
    add_special_tokens=True,
    max_length=max_len,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt')
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      print(final_output[0].index(max(final_output[0])))
      print(final_output[0])
      result=inx_class_dict[final_output[0].index(max(final_output[0]))]
  return result



In [36]:
answerlist=[]
for each in val_df['abstract']:
  output=predict(trained_model,each)
  answerlist.append(output)


Streaming output truncated to the last 5000 lines.
5
[0.0013835435966029763, 0.0009514245903119445, 0.0023728543892502785, 0.0014051352627575397, 0.26941776275634766, 0.32479938864707947, 0.004343744833022356, 0.0005249282112345099, 0.0014793981099501252, 0.0002037074591498822, 0.008410985581576824, 0.0027754742186516523, 0.0010846942896023393, 0.005733409896492958, 0.0006104285712353885, 0.0009354089270345867, 0.3211914300918579, 0.0036642951890826225, 0.0015536574646830559]
13
[0.0008244271157309413, 0.0010107774287462234, 0.0019321992294862866, 0.0020646112971007824, 0.005056381691247225, 0.0013427147641777992, 0.0018769213929772377, 0.0005379125359468162, 0.0010274006053805351, 0.0009354798821732402, 0.0035483171232044697, 0.0010480069322511554, 0.0009244961547665298, 0.9815350770950317, 0.0018215776653960347, 0.0020538647659122944, 0.002347704488784075, 0.0016071415739133954, 0.0022744927555322647]
10
[0.0024738984648138285, 0.0036310444120317698, 0.0038530072197318077, 0.01269342

In [37]:
answerlist

['Crystal structure',
 'Crystal structure',
 'Crystal structure',
 'Particulate matter',
 'Insulators',
 'Insulators',
 'Insulators',
 'Fibers',
 'Composites',
 'Plastics',
 'Insulators',
 'Granular materials',
 'Composites',
 'Crystal structure',
 'Coating materials',
 'Particulate matter',
 'Solvents',
 'Crystal structure',
 'Granular materials',
 'Particulate matter',
 'Dyes and pigments',
 'Crystal structure',
 'Crystal structure',
 'Alloys',
 'Fibers',
 'Composites',
 'Plastics',
 'Particulate matter',
 'Particulate matter',
 'Solvents',
 'Crystal structure',
 'Crystal structure',
 'Coating materials',
 'Granular materials',
 'Particulate matter',
 'Granular materials',
 'Solvents',
 'Crystal structure',
 'Granular materials',
 'Particulate matter',
 'Particulate matter',
 'Crystal structure',
 'Plastics',
 'Anode materials',
 'Dyes and pigments',
 'Dyes and pigments',
 'Vinyl',
 'Dyes and pigments',
 'Plastics',
 'Crystal structure',
 'Particulate matter',
 'Solvents',
 'Solvents

In [38]:
 val_df['abstract']

0        The molecular and crystal structure of the hyd...
1        The first [2]silacobaltocenophane, [(MeSi)(η-C...
2        Rational drug design depends on the knowledge ...
3        Electrophoretically induced aggregation of Au ...
4        Dielectric relaxation of the ionic liquid, 1-e...
                               ...                        
17995    The rehydration of thermoresponsive poly(monom...
17996    The experimental solubility data of fenofibrat...
17997    Niobium pentoxide, NbO, exists in many polymor...
17998    Pure and -nitroaniline (O-NA) doped benzil sin...
17999    We describe the characterization of the hydrod...
Name: abstract, Length: 18000, dtype: object

In [39]:
val_df

,abstract,Alloys,Anode materials,Coating materials,Composites,Crystal structure,Crystals,Dyes and pigments,Electrical conductivity,Fibers,Foams,Granular materials,Insulators,Layered materials,Particulate matter,Plastics,Porous materials,Solvents,Thin films,Vinyl
0,The molecular and crystal structure of the hyd...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The first [2]silacobaltocenophane, [(MeSi)(η-C...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rational drug design depends on the knowledge ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Electrophoretically induced aggregation of Au ...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,"Dielectric relaxation of the ionic liquid, 1-e...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,The rehydration of thermoresponsive poly(monom...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
17996,The experimental solubility data of fenofibrat...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
17997,"Niobium pentoxide, NbO, exists in many polymor...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
17998,Pure and -nitroaniline (O-NA) doped benzil sin...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
df_test=pd.read_csv("drive/MyDrive/test_data.csv")
answerlist_test=[]

for each in df_test['abstract']:
  output=predict(trained_model,each)
  answerlist_test.append(output)

Streaming output truncated to the last 5000 lines.
14
[0.0009064366458915174, 0.004386972635984421, 0.10783378779888153, 0.07336369901895523, 0.001352973165921867, 0.0016600381350144744, 0.0029544199351221323, 0.0016231425106525421, 0.006540488451719284, 0.0013142263051122427, 0.0041344016790390015, 0.009559407830238342, 0.013759121298789978, 0.0025255626533180475, 0.5950308442115784, 0.005105887074023485, 0.0027058636769652367, 0.2634091079235077, 0.001585078309290111]
14
[0.0009760355460457504, 0.001990330871194601, 0.0026694999542087317, 0.0023832812439650297, 0.0015425774035975337, 0.0007047500694170594, 0.0026687635108828545, 0.003207847708836198, 0.0016281093703582883, 0.001381526468321681, 0.01274710800498724, 0.0014658001018688083, 0.00266824197024107, 0.003998050466179848, 0.9344160556793213, 0.0029123760759830475, 0.018039418384432793, 0.00960749201476574, 0.0021742978133261204]
14
[0.0005627782666124403, 0.000992061453871429, 0.0020428376737982035, 0.0009518646984361112, 0.0

In [63]:
answerlist_df = pd.DataFrame(answerlist_test)
answerlist_df.insert(0, 'test_id', range(1, len(answerlist_df)+1))
answerlist_df = answerlist_df.rename(columns = {0: "label"})
answerlist_df

,test_id,label
0,1,Foams
1,2,Fibers
2,3,Coating materials
3,4,Coating materials
4,5,Crystals
...,...,...
9995,9996,Crystal structure
9996,9997,Porous materials
9997,9998,Granular materials
9998,9999,Plastics


In [52]:
answerlist_df

,label
0,Foams
1,Fibers
2,Coating materials
3,Coating materials
4,Crystals
...,...
9995,Crystal structure
9996,Porous materials
9997,Granular materials
9998,Plastics


In [64]:
answerlist_df

,test_id,label
0,1,Foams
1,2,Fibers
2,3,Coating materials
3,4,Coating materials
4,5,Crystals
...,...,...
9995,9996,Crystal structure
9996,9997,Porous materials
9997,9998,Granular materials
9998,9999,Plastics


In [43]:
from sklearn.metrics import accuracy_score

In [65]:
answerlist_df.to_csv('drive/MyDrive/pred_labels.csv',index=False)